In [ ]:
import sys as _sys
import os

current_path = os.path.abspath(os.getcwd())

split = current_path.split("inverse_geometric_locomotion")
if len(split)<2:
    print("Please rename the repository 'inverse_geometric_locomotion'")
    raise ValueError
path_to_python_scripts = os.path.join(split[0], "inverse_geometric_locomotion/python/")
path_to_notifications = os.path.join(split[0], "inverse_geometric_locomotion/notebooks/notifications/")
path_to_settings = os.path.join(split[0], "inverse_geometric_locomotion/python/figures/")
path_to_cubic_splines = os.path.join(split[0], "inverse_geometric_locomotion/ext/torchcubicspline/")
path_to_output = os.path.join(split[0], "inverse_geometric_locomotion/output/")
path_to_data = path_to_output
path_to_save = os.path.join(path_to_output, "snake_confined")

if not os.path.exists(path_to_save):
    os.makedirs(path_to_save)

_sys.path.insert(0, path_to_python_scripts)
_sys.path.insert(0, path_to_settings)
_sys.path.insert(0, path_to_cubic_splines)

In [ ]:
import json
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import shutil
import torch

from obstacle_implicits import BoxImplicit, ComplementaryImplicit
from vis_utils import produce_video_from_path, print_json_data
from vis_utils_snake import plot_animated_snake

blue = tuple((np.array([59.0, 130.0, 219.0]) / 255.0).tolist())

## Load data

This notebook can be used to read files generated from executing `python/experiments/snake_confined.py`. The following list assumes you have executed the experiment script successfully with the flag `--trial_number=0`. 

As you run more experiments with different `trial_number`, you can add the output path to the list `exp_file_names` with the corresponding file locations. The naming convention for `.json` files is `*_opt_{trial_number}.json`, where `{trial_number}` always has two digits.

In [ ]:
exp_file_names = [
    "snake_confined/snake_confined_opt_00.json",
]

trial_numbers = [int(fn.split('_')[-1].split('.')[0]) for fn in exp_file_names]
exp_names = ["snake_confined_{:02d}".format(tn) for tn in trial_numbers]

list_js_loads = []
for exp_file_name in exp_file_names:
    with open(os.path.join(path_to_data, exp_file_name)) as jsonFile:
        js_load = json.load(jsonFile)

    print(exp_file_name)
    print_json_data(js_load)
    list_js_loads.append(js_load)

print("\nLoaded {} experiments.".format(len(list_js_loads)))

## Plot snake trajectory in 2D

Check if the snake completed its turn in a box. Note that the box is shown even if the collision avoidance term was turned off.

You can select which experiment to plot from the list in the previous cell `exp_file_names` by changing `exp_id` accordingly. By default, the first element of the list is used.

In [ ]:
exp_id = 0 # You can change this to plot a different experiment
js_load = list_js_loads[exp_id]
show_obstacle = True

pos = np.array(js_load['pos'])
n_ts = pos.shape[0]
g = np.array(js_load['g'])
gt = np.array(js_load['optimization_settings']['gt'])
params_opt = np.array(js_load['optimization_settings']['params_opt'])
boxsize = js_load['optimization_settings']['boxsize']

box_params = torch.tensor([0.0, 0.0, 0.0, boxsize, boxsize, 1.0])
box_implicit = BoxImplicit(box_params)
obstacle = ComplementaryImplicit(box_implicit)

alphas = np.linspace(0.1, 1.0, n_ts)
fig = plt.figure(figsize=(10, 6))
gs = fig.add_gridspec(1, 1)
ax_tmp = fig.add_subplot(gs[0, 0])
for id_step in range(n_ts):
    ax_tmp.plot(pos[id_step, :, 0], pos[id_step, :, 1], lw=3.0, c=blue, alpha=alphas[id_step], zorder=0)
    ax_tmp.scatter(g[id_step, 4], g[id_step, 5], marker='x', s=30.0, c=blue, alpha=alphas[id_step], zorder=0)
ax_tmp.scatter(gt[4], gt[5], marker='o', s=30.0, c='tab:orange', alpha=1.0, zorder=1)

if show_obstacle:
    box_mpl = Rectangle((box_params[0] - box_params[3], box_params[1] - box_params[4]), 2.0 * box_params[3], 2.0 * box_params[4], facecolor=(0.0, 0.0, 0.0, 0.0), edgecolor='k', lw=2.0, zorder=-1)
    ax_tmp.add_patch(box_mpl)

    n_plot = 1000
    n_levels = 15
    x_plot = torch.linspace(ax_tmp.get_xlim()[0], ax_tmp.get_xlim()[1], n_plot)
    y_plot = torch.linspace(ax_tmp.get_ylim()[0], ax_tmp.get_ylim()[1], n_plot)
    xyz_plot = torch.stack([
        torch.tile(x_plot, dims=(n_plot,)),
        torch.repeat_interleave(y_plot, repeats=n_plot, dim=0),
        torch.zeros(size=(n_plot*n_plot,))
    ], dim=1)

    sdfs = obstacle.evaluate_implicit_function(xyz_plot).reshape(n_plot, n_plot)

    min_sdf, max_sdf = torch.min(sdfs), torch.max(sdfs)
    max_abs_sdf = max(torch.abs(min_sdf), torch.abs(max_sdf))
    levels = np.linspace(-max_abs_sdf, max_abs_sdf, n_levels)
    ax_tmp.contourf(x_plot, y_plot, sdfs, levels=levels, cmap='coolwarm', zorder=-2)
    ax_tmp.contour(x_plot, y_plot, sdfs, levels=[0.0], colors='k', linewidths=3.0, zorder=-1)

ax_tmp.set_aspect('equal')
plt.axis('off')
plt.show()

## Produce an animation of the gait

Animate the previous plot and save it in the same folder in which you saved the output `.json` files.

You can list all the experiments you would like to generate the trajectories from by filling out `exp_id_list` with all the corresponding experiments indices in `exp_file_names`. By default, it will only generate the animation for the first experiment in the list.

In [ ]:
exp_id_list = [0] # Update this list with the indices of the experiments you want to animate

path_to_images_anim = os.path.join(path_to_save, "images_registered")
arrow_params = {
    "length": 0.05,
    "width": 0.02,
}

xy_lim = np.array([
    [-1.0, 1.0],
    [-1.0, 1.0],
])

for exp_id in exp_id_list:
    print("Producing animation for experiment: {}".format(exp_file_names[exp_id]))

    js_load = list_js_loads[exp_id]
    exp_name = exp_names[exp_id]
    pos_plot = np.array(js_load['pos'])
    g_plot = np.array(js_load['g'])
    broken_joint_ids = []

    # clear existing images
    if os.path.exists(path_to_images_anim):
        shutil.rmtree(path_to_images_anim)
    os.makedirs(path_to_images_anim)

    plot_animated_snake(
        pos_plot, path_to_images_anim,
        g=g_plot, gt=None, gcp=None,
        broken_joint_ids=broken_joint_ids, obstacle=obstacle,
        exponent=1.0, xy_lim=xy_lim, 
        show_orientation=False, show_snake_trail=False, 
        show_g_trail=True, show_g_start=True, show_joints=False,
        arrow_params=arrow_params,
    )

    fn_pattern = os.path.join(path_to_images_anim, "step_%05d.png")
    produce_video_from_path(
        fn_pattern, path_to_save, 
        "{}_pos.mp4".format(exp_name), overwrite_anim=True, transparent=False
    )